In [4]:
# from sst and rainfall ---> rainfall
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt 
import tensorflow.compat.v1 as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv3D, Conv2D
from keras.layers import ConvLSTM2D
from keras.layers import Attention
from keras.layers import Input
from keras.models import Model
from keras.layers import Dropout, Lambda
from keras import optimizers
from keras.layers import Conv2DTranspose, TimeDistributed
from keras import callbacks
import xarray as xr

# Define global variables
lead_time = 2
save_path = "/home/cccr/roxy/matin/MTech_project/model/Conv-LSTM/7in3out/"
model_path = save_path + "ConvLstm_sst0rf_rf_2bob.h5"
add_data = "/home/cccr/roxy/matin/MTech_project/data/"

def preprocess_data(sequence, n_steps, channels):
    """
    Function to preprocess the data and prepare it for model training.
    
    Parameters:
        sequence (xarray.Dataset): Data sequence to be preprocessed.
        n_steps (int): Number of time steps in the input sequence.
        channels (int): Number of channels in the input sequence.
    
    Returns:
        tuple: A tuple containing the preprocessed input sequence (X) and output sequence (y).
    """
    # Select data for Jun, Jul, Aug, Sept
    sequence = sequence.sel(time=slice("1998-01-01","2016-12-31"))
    sequence = sequence.where(sequence.time.dt.month.isin([6,7,8,9]), drop=True)
    max = sequence.max()
    min = sequence.min()
    
    # Max-min normalization
    sequence = (sequence-min)/(max - min)
    sequence = np.array(sequence)
    
    # Exponential Space Transform
    valid_pts = np.where(sequence==sequence)
    sequence[valid_pts] = np.exp(sequence[valid_pts])
    sequence[valid_pts] = np.power(sequence[valid_pts],7)
    invalid_pts = np.where(sequence != sequence)
    sequence[invalid_pts] = 0
    
    # Prepare input and output sequences
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix + lead_time > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix + lead_time]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


def create_input(channels, n_steps):
    """
    Function to create the input for the model.
    
    Parameters:
        channels (list): List of channels to be used in the input.
        n_steps (int): Number of time steps in the input sequence.
        
    Returns:
        numpy.ndarray: The input for the model.
    """
    stack = []
    for i in range(len(channels)):
        channel_data = xr.open_dataarray(f"/home/cccr/roxy/matin/MTech_project/data/{channels[i]}")
        input, _ = preprocess_data(channel_data, n_steps=n_steps, channels=channels[i])
        input = np.expand_dims(input, axis=2)
        print(f"Adding channel {i} with shape: {input.shape}")
        stack.append(input)
    out = np.dstack(stack)
    return out


channels = ["FilteredrfBOB_0lag.nc","FilteredSSTBOB_0.nc"]
steps = 7

# Create input for the model
input = create_input(channels, steps)

# Preprocess target data
channel_data = xr.open_dataarray(add_data + "FilteredrfBOB_0lag.nc")
_, target = preprocess_data(channel_data, n_steps=steps, channels=channels[-1])
target = np.expand_dims(target, axis=1)

# Roll axes for input and target
x = np.rollaxis(input, 4, 2)
x = np.rollaxis(x, 4, 2)

y = np.rollaxis(target, 3, 1)
y = np.rollaxis(y, 3, 1)

# Assert that input and target shapes are consistent
assert input.shape[0] == target.shape[0]
assert input.shape[-1] == target.shape[-1]
assert input.shape[-2] == target.shape[-2]


# Print shape of input and target
print("INPUT SHAPE --> ", input.shape)
print("TARGET SHAPE --> ", target.shape)

# Delete original input and target variables to free up memory
del input
del target

# Perform additional processing or modeling steps as needed
seq = tf.keras.Sequential()

# Add layers to the model
seq.add(ConvLSTM2D(filters=4, kernel_size=(3,3), padding='same', input_shape=(7,25,39,2), return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=8, kernel_size=(3,3), padding='same', return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=8, kernel_size=(3,3), padding='same', return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=16, kernel_size=(3,3), padding='same', return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=16, kernel_size=(3,3), padding='same', return_sequences=False, data_format='channels_last'))
seq.add(Conv2D(filters=15, kernel_size=(3,3), activation='relu', padding='same', data_format='channels_last'))
seq.add(Conv2D(filters=1, kernel_size=(3,3), activation='relu', padding='same', data_format='channels_last'))
# Create the query, value, and key inputs
query_input = Input(shape=(25,39,2), name='query_input')
value_input = Input(shape=(25,39,2), name='value_input')
key_input = Input(shape=(25,39,2), name='key_input')

# Pass the inputs to the Attention layer
attention_output = Attention(name='attention')([query_input, value_input, key_input])
Adam = tf.keras.optimizers.Adam(learning_rate=10**-4)
seq.compile(loss='mean_squared_error', optimizer=Adam, metrics=['mae'])

print(seq.summary())

# Define the number of training epochs
n_epochs = 200
logdir = save_path
# Define the TensorBoard callback for monitoring training progress
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=10, batch_size=10, write_graph=True)

# Define the ReduceLROnPlateau callback for reducing the learning rate when the model plateaus
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=20, min_lr=10**-20)

# Define the ModelCheckpoint callback for saving the best model based on validation mean absolute error
checkpoint_callback_mae = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_mae',
                                                             mode='min', save_best_only=True)

# Define the ModelCheckpoint callback for saving the best model based on validation loss
checkpoint_callback_loss = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss',
                                                              mode='min', save_best_only=True)

# Define the EarlyStopping callback for stopping training when the model stops improving
early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True)

# Define the TerminateOnNaN callback for terminating the training process if there is NaN value in any of the output
terminate_callback = tf.keras.callbacks.TerminateOnNaN()

# Train the model using the defined callbacks
history = seq.fit(x, y, epochs=n_epochs, validation_split=0.1,
                  batch_size=50, callbacks=[early_stop_callback,
                  checkpoint_callback_mae, checkpoint_callback_loss,
                  tb_callback, lr_callback, terminate_callback])
#Save the trained model
seq.save(model_path)

#Save the training history

np.save(f'{save_path}ConvLstm_sst0rf_rf_2bob.npy', history.history)               
              

Adding channel 0 with shape: (2309, 7, 1, 25, 39)
Adding channel 1 with shape: (2309, 7, 1, 25, 39)
INPUT SHAPE -->  (2309, 7, 2, 25, 39)
TARGET SHAPE -->  (2309, 1, 25, 39)
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_5 (ConvLSTM2D)  (None, 7, 25, 39, 4)      880       
                                                                 
 conv_lstm2d_6 (ConvLSTM2D)  (None, 7, 25, 39, 8)      3488      
                                                                 
 conv_lstm2d_7 (ConvLSTM2D)  (None, 7, 25, 39, 8)      4640      
                                                                 
 conv_lstm2d_8 (ConvLSTM2D)  (None, 7, 25, 39, 16)     13888     
                                                                 
 conv_lstm2d_9 (ConvLSTM2D)  (None, 25, 39, 16)        18496     
                                                                 
 conv2d_2 (C

Epoch 43/200
42/42 [==============================] - 42s 1s/step - loss: 1037.1919 - mae: 20.9549 - val_loss: 924.4783 - val_mae: 19.1778 - lr: 1.0000e-04
Epoch 44/200
42/42 [==============================] - 37s 887ms/step - loss: 1037.0833 - mae: 20.8924 - val_loss: 924.2814 - val_mae: 19.1803 - lr: 1.0000e-04
Epoch 45/200
42/42 [==============================] - 41s 972ms/step - loss: 1036.8243 - mae: 20.9132 - val_loss: 924.1006 - val_mae: 19.1856 - lr: 1.0000e-04
Epoch 46/200
42/42 [==============================] - 38s 896ms/step - loss: 1036.7991 - mae: 20.9092 - val_loss: 923.6942 - val_mae: 19.1450 - lr: 1.0000e-04
Epoch 47/200
42/42 [==============================] - 36s 849ms/step - loss: 1036.7034 - mae: 20.9485 - val_loss: 923.3699 - val_mae: 19.1173 - lr: 1.0000e-04
Epoch 48/200
42/42 [==============================] - 42s 1s/step - loss: 1036.3923 - mae: 20.9337 - val_loss: 923.4326 - val_mae: 19.1778 - lr: 1.0000e-04
Epoch 49/200
42/42 [==============================] 

42/42 [==============================] - 34s 815ms/step - loss: 1029.2583 - mae: 20.7919 - val_loss: 914.0493 - val_mae: 19.0051 - lr: 1.0000e-04
Epoch 95/200
42/42 [==============================] - 34s 799ms/step - loss: 1029.0319 - mae: 20.7672 - val_loss: 913.8217 - val_mae: 18.9879 - lr: 1.0000e-04
Epoch 96/200
42/42 [==============================] - 35s 830ms/step - loss: 1028.8665 - mae: 20.7967 - val_loss: 913.7665 - val_mae: 19.0153 - lr: 1.0000e-04
Epoch 97/200
42/42 [==============================] - 35s 838ms/step - loss: 1029.1301 - mae: 20.8353 - val_loss: 913.3449 - val_mae: 18.9411 - lr: 1.0000e-04
Epoch 98/200
42/42 [==============================] - 34s 815ms/step - loss: 1028.5894 - mae: 20.7412 - val_loss: 913.3864 - val_mae: 19.0014 - lr: 1.0000e-04
Epoch 99/200
42/42 [==============================] - 34s 819ms/step - loss: 1028.6034 - mae: 20.8340 - val_loss: 913.4133 - val_mae: 19.0414 - lr: 1.0000e-04
Epoch 100/200
42/42 [==============================] - 34s 

42/42 [==============================] - 34s 803ms/step - loss: 1022.0961 - mae: 20.7024 - val_loss: 906.0086 - val_mae: 18.8527 - lr: 1.0000e-04
Epoch 146/200
42/42 [==============================] - 35s 827ms/step - loss: 1022.5036 - mae: 20.6563 - val_loss: 906.6929 - val_mae: 19.0177 - lr: 1.0000e-04
Epoch 147/200
42/42 [==============================] - 35s 824ms/step - loss: 1021.9520 - mae: 20.7392 - val_loss: 905.9300 - val_mae: 18.9028 - lr: 1.0000e-04
Epoch 148/200
42/42 [==============================] - 35s 824ms/step - loss: 1021.8475 - mae: 20.6003 - val_loss: 905.5890 - val_mae: 18.8464 - lr: 1.0000e-04
Epoch 149/200
42/42 [==============================] - 34s 811ms/step - loss: 1021.7117 - mae: 20.7502 - val_loss: 905.8625 - val_mae: 18.9427 - lr: 1.0000e-04
Epoch 150/200
42/42 [==============================] - 34s 807ms/step - loss: 1021.9720 - mae: 20.5523 - val_loss: 905.6797 - val_mae: 18.9326 - lr: 1.0000e-04
Epoch 151/200
42/42 [==============================] -

42/42 [==============================] - 37s 886ms/step - loss: 1016.7417 - mae: 20.6886 - val_loss: 899.8438 - val_mae: 18.7805 - lr: 1.0000e-04
Epoch 197/200
42/42 [==============================] - 36s 854ms/step - loss: 1016.5071 - mae: 20.5329 - val_loss: 899.9520 - val_mae: 18.8334 - lr: 1.0000e-04
Epoch 198/200
42/42 [==============================] - 37s 879ms/step - loss: 1016.2105 - mae: 20.6112 - val_loss: 899.9920 - val_mae: 18.8608 - lr: 1.0000e-04
Epoch 199/200
42/42 [==============================] - 37s 886ms/step - loss: 1016.2108 - mae: 20.6879 - val_loss: 899.5267 - val_mae: 18.7737 - lr: 1.0000e-04
Epoch 200/200
42/42 [==============================] - 34s 821ms/step - loss: 1016.2034 - mae: 20.5704 - val_loss: 899.8089 - val_mae: 18.8617 - lr: 1.0000e-04


In [5]:
def preprocess_data(sequence, n_steps, channels):
    """
    Function to preprocess the data and prepare it for model training.
    
    Parameters:
        sequence (xarray.Dataset): Data sequence to be preprocessed.
        n_steps (int): Number of time steps in the input sequence.
        channels (int): Number of channels in the input sequence.
    
    Returns:
        tuple: A tuple containing the preprocessed input sequence (X) and output sequence (y).
    """
    # Select data for Jun, Jul, Aug, Sept
    sequence = sequence.sel(time=slice("2017-01-01","2018-12-31"))
    sequence = sequence.where(sequence.time.dt.month.isin([6,7,8,9]), drop=True)
    max = sequence.max()
    min = sequence.min()
    
    # Max-min normalization
    sequence = (sequence-min)/(max - min)
    sequence = np.array(sequence)
    
    # Exponential Space Transform
    valid_pts = np.where(sequence==sequence)
    sequence[valid_pts] = np.exp(sequence[valid_pts])
    sequence[valid_pts] = np.power(sequence[valid_pts],7)
    invalid_pts = np.where(sequence != sequence)
    sequence[invalid_pts] = 0
    
    # Prepare input and output sequences
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix + lead_time > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix + lead_time]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


def create_input(channels, n_steps):
    """
    Function to create the input for the model.
    
    Parameters:
        channels (list): List of channels to be used in the input.
        n_steps (int): Number of time steps in the input sequence.
        
    Returns:
        numpy.ndarray: The input for the model.
    """
    stack = []
    for i in range(len(channels)):
        channel_data = xr.open_dataarray(f"/home/cccr/roxy/matin/MTech_project/data/{channels[i]}")
        input, _ = preprocess_data(channel_data, n_steps=n_steps, channels=channels[i])
        input = np.expand_dims(input, axis=2)
        print(f"Adding channel {i} with shape: {input.shape}")
        stack.append(input)
    out = np.dstack(stack)
    return out


channels = ["FilteredrfBOB_0lag.nc","FilteredSSTBOB_0.nc"]
steps = 7

# Create input for the model
input = create_input(channels, steps)

# Preprocess target data
channel_data = xr.open_dataarray(add_data + "FilteredrfBOB_0lag.nc")
_, target = preprocess_data(channel_data, n_steps=steps, channels=channels[-1])
target = np.expand_dims(target, axis=1)

# Roll axes for input and target
x = np.rollaxis(input, 4, 2)
x = np.rollaxis(x, 4, 2)

y = np.rollaxis(target, 3, 1)
y = np.rollaxis(y, 3, 1)

# Assert that input and target shapes are consistent
assert input.shape[0] == target.shape[0]
assert input.shape[-1] == target.shape[-1]
assert input.shape[-2] == target.shape[-2]


# Print shape of input and target
print("INPUT SHAPE --> ", input.shape)
print("TARGET SHAPE --> ", target.shape)

# Delete original input and target variables to free up memory
del input
del target


Adding channel 0 with shape: (235, 7, 1, 25, 39)
Adding channel 1 with shape: (235, 7, 1, 25, 39)
INPUT SHAPE -->  (235, 7, 2, 25, 39)
TARGET SHAPE -->  (235, 1, 25, 39)


In [6]:
model = tf.keras.models.load_model(model_path)
pred = model.predict(x)

actual = y.flatten()
prediction = pred.flatten()
corr = np.corrcoef(actual,prediction)
corr

8/8 [==============================] - 4s 92ms/step


array([[ 1.        , -0.05160796],
       [-0.05160796,  1.        ]])